In [1]:
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()
# 2021.09.20 insert 에러
cursor = mydb.cursor(prepared=True)
citycode = '31040'

query1 = ("select citycode from public_busctycodelist where citycode >= {}".format(citycode))

cursor.execute(query1)

location_list1 = []

for a in cursor:
    location_list1.append(a)
    
b_cnt = 0
for z in location_list1:
    if b_cnt == 1:
        break
        
    location = z
    query2 = ("select distinct routeid from public_busroute_info where citycode = '{}'".format(location[0]))
    cursor.execute(query2)

    location_list2 = []

    for b in cursor:
        location_list2.append(b)

    now = datetime.now()
    nowDate = now.strftime('%Y%m%d')

    cnt = 1
    for i in location_list2:
        routeId = i

        characters = "(',')"

        join_data = ''.join( x for x in routeId if x not in characters)
        print("조회날짜 : {} | location : {} | cnt : {} | start {} public_busroute_info".format(nowDate,location[0],cnt,join_data))

        url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteAcctoThrghSttnList'
        queryParams = '?' + urlencode({
            quote_plus('serviceKey') : public_serviceKey(),
            quote_plus('cityCode') : citycode,
            quote_plus('pageNo') : '1',
            quote_plus('numOfRows') : '10000',
            quote_plus('routeId') : join_data
        })

        response = requests.get(url + queryParams).text.encode('utf-8')
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        rows = xmlobj.findAll('item')
        resultCode = xmlobj.findAll('resultCode')
        if resultCode[0].text == '99':
            print("location {} | {} 번째부터 조회수 제한이 걸렸습니다.".format(location[0],join_data))
            mydb.close()
            b_cnt += 1
            break
        cnt += 1
        for j in rows:
            item = j
            lat = item.findAll('gpslati')
            lng = item.findAll('gpslong')
            nodeid = item.findAll('nodeid')
            nodenm = item.findAll('nodenm')
            nodeno = item.findAll('nodeno')
            nodeord = item.findAll('nodeord')
            routeid = item.findAll('routeid')
            updowncd = item.findAll('updowncd')
            try:
                if updowncd[0].text == '0':
                    updowncd = '상행'
                elif updowncd[0].text == '1':
                    updowncd = '하행'
            except IndexError:
                updowncd = None

            try:
                sql = "INSERT INTO public_busroute_detail (lat, lng, nodeid, nodenm, nodeno, nodeord, routeid, updowncd,citycode) VALUES (?,?,?,?,?,?,?,?,?)"
                val = (lat[0].text, lng[0].text, nodeid[0].text, nodenm[0].text, nodeno[0].text, nodeord[0].text, routeid[0].text, updowncd,citycode)
            except IndexError:
                sql = "INSERT INTO public_busroute_detail (lat, lng, nodeid, nodenm, nodeno, nodeord, routeid, updowncd,citycode) VALUES (?,?,?,?,?,?,?,?,?)"
                val = (lat[0].text, lng[0].text, nodeid[0].text, nodenm[0].text, None, nodeord[0].text, routeid[0].text, updowncd,citycode)

            cursor.execute(sql, val)
            mydb.commit()

조회날짜 : 20210920 | location : 31040 | cnt : 1 | start GGB208000014 public_busroute_info
조회날짜 : 20210920 | location : 31040 | cnt : 2 | start GGB208000018 public_busroute_info
조회날짜 : 20210920 | location : 31040 | cnt : 3 | start GGB208000025 public_busroute_info
조회날짜 : 20210920 | location : 31040 | cnt : 4 | start GGB208000039 public_busroute_info
조회날짜 : 20210920 | location : 31040 | cnt : 5 | start GGB208000041 public_busroute_info
조회날짜 : 20210920 | location : 31040 | cnt : 6 | start GGB208000001 public_busroute_info
조회날짜 : 20210920 | location : 31040 | cnt : 7 | start GGB208000002 public_busroute_info
조회날짜 : 20210920 | location : 31040 | cnt : 8 | start GGB208000004 public_busroute_info
조회날짜 : 20210920 | location : 31040 | cnt : 9 | start GGB208000005 public_busroute_info
조회날짜 : 20210920 | location : 31040 | cnt : 10 | start GGB208000006 public_busroute_info
조회날짜 : 20210920 | location : 31040 | cnt : 11 | start GGB208000007 public_busroute_info
조회날짜 : 20210920 | location : 31040 | cnt 

조회날짜 : 20210920 | location : 31060 | cnt : 9 | start GGB213000006 public_busroute_info
조회날짜 : 20210920 | location : 31060 | cnt : 10 | start GGB213000007 public_busroute_info
조회날짜 : 20210920 | location : 31060 | cnt : 11 | start GGB213000008 public_busroute_info
조회날짜 : 20210920 | location : 31060 | cnt : 12 | start GGB213000011 public_busroute_info
조회날짜 : 20210920 | location : 31060 | cnt : 13 | start GGB213000014 public_busroute_info
조회날짜 : 20210920 | location : 31060 | cnt : 14 | start GGB213000003 public_busroute_info
조회날짜 : 20210920 | location : 31060 | cnt : 15 | start GGB213000010 public_busroute_info
조회날짜 : 20210920 | location : 31060 | cnt : 16 | start GGB213000017 public_busroute_info
조회날짜 : 20210920 | location : 31060 | cnt : 17 | start GGB208000066 public_busroute_info
조회날짜 : 20210920 | location : 31060 | cnt : 18 | start GGB213000009 public_busroute_info
조회날짜 : 20210920 | location : 31060 | cnt : 19 | start GGB213000026 public_busroute_info
조회날짜 : 20210920 | location : 3107

조회날짜 : 20210920 | location : 31070 | cnt : 84 | start GGB214000108 public_busroute_info
조회날짜 : 20210920 | location : 31070 | cnt : 85 | start GGB214000115 public_busroute_info
조회날짜 : 20210920 | location : 31070 | cnt : 86 | start GGB214000153 public_busroute_info
조회날짜 : 20210920 | location : 31070 | cnt : 87 | start GGB214000012 public_busroute_info
조회날짜 : 20210920 | location : 31070 | cnt : 88 | start GGB214000067 public_busroute_info
조회날짜 : 20210920 | location : 31070 | cnt : 89 | start GGB214000147 public_busroute_info
조회날짜 : 20210920 | location : 31070 | cnt : 90 | start GGB214000182 public_busroute_info
조회날짜 : 20210920 | location : 31070 | cnt : 91 | start GGB214000070 public_busroute_info
조회날짜 : 20210920 | location : 31070 | cnt : 92 | start GGB214000079 public_busroute_info
조회날짜 : 20210920 | location : 31070 | cnt : 93 | start GGB214000162 public_busroute_info
조회날짜 : 20210920 | location : 31070 | cnt : 94 | start GGB214000168 public_busroute_info
조회날짜 : 20210920 | location : 310

조회날짜 : 20210920 | location : 31080 | cnt : 71 | start GGB215000216 public_busroute_info
조회날짜 : 20210920 | location : 31080 | cnt : 72 | start GGB215000223 public_busroute_info
조회날짜 : 20210920 | location : 31080 | cnt : 73 | start GGB215000028 public_busroute_info
조회날짜 : 20210920 | location : 31080 | cnt : 74 | start GGB215000222 public_busroute_info
조회날짜 : 20210920 | location : 31080 | cnt : 75 | start GGB215000015 public_busroute_info
조회날짜 : 20210920 | location : 31080 | cnt : 76 | start GGB215000021 public_busroute_info
조회날짜 : 20210920 | location : 31080 | cnt : 77 | start GGB215000088 public_busroute_info
조회날짜 : 20210920 | location : 31080 | cnt : 78 | start GGB215000102 public_busroute_info
조회날짜 : 20210920 | location : 31080 | cnt : 79 | start GGB215000133 public_busroute_info
조회날짜 : 20210920 | location : 31080 | cnt : 80 | start GGB215000149 public_busroute_info
조회날짜 : 20210920 | location : 31080 | cnt : 81 | start GGB215000150 public_busroute_info
조회날짜 : 20210920 | location : 310

조회날짜 : 20210920 | location : 31100 | cnt : 24 | start GGB219000021 public_busroute_info
조회날짜 : 20210920 | location : 31100 | cnt : 25 | start GGB229000035 public_busroute_info
조회날짜 : 20210920 | location : 31100 | cnt : 26 | start GGB218000112 public_busroute_info
조회날짜 : 20210920 | location : 31100 | cnt : 27 | start GGB218000001 public_busroute_info
조회날짜 : 20210920 | location : 31100 | cnt : 28 | start GGB219000008 public_busroute_info
조회날짜 : 20210920 | location : 31100 | cnt : 29 | start GGB232000004 public_busroute_info
조회날짜 : 20210920 | location : 31100 | cnt : 30 | start GGB229000018 public_busroute_info
조회날짜 : 20210920 | location : 31100 | cnt : 31 | start GGB232000003 public_busroute_info
조회날짜 : 20210920 | location : 31100 | cnt : 32 | start GGB218000006 public_busroute_info
조회날짜 : 20210920 | location : 31100 | cnt : 33 | start GGB229000071 public_busroute_info
조회날짜 : 20210920 | location : 31100 | cnt : 34 | start GGB218000018 public_busroute_info
조회날짜 : 20210920 | location : 311

...

조회날짜 : 20210920 | location : 31140 | cnt : 64 | start GGB224000054 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 65 | start GGB233000362 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 66 | start GGB233000364 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 67 | start GGB224000040 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 68 | start GGB233000363 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 69 | start GGB233000365 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 70 | start GGB224000058 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 71 | start GGB216000040 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 72 | start GGB224000033 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 73 | start GGB224000059 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 74 | start GGB225000004 public_busroute_info
조회날짜 : 20210920 | location : 311

조회날짜 : 20210920 | location : 31140 | cnt : 159 | start GGB234000882 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 160 | start GGB234000884 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 161 | start GGB234001271 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 162 | start GGB228000238 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 163 | start GGB228000391 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 164 | start GGB228000394 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 165 | start GGB228000012 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 166 | start GGB228000055 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 167 | start GGB228000082 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 168 | start GGB228000133 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 169 | start GGB228000136 public_busroute_info
조회날짜 : 20210920 | loc

조회날짜 : 20210920 | location : 31140 | cnt : 254 | start GGB229000242 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 255 | start GGB229000243 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 256 | start GGB229000244 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 257 | start GGB229000247 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 258 | start GGB229000249 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 259 | start GGB229000250 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 260 | start GGB229000252 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 261 | start GGB229000255 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 262 | start GGB229000261 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 263 | start GGB229000262 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 264 | start GGB229000268 public_busroute_info
조회날짜 : 20210920 | loc

조회날짜 : 20210920 | location : 31140 | cnt : 350 | start GGB234001194 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 351 | start GGB234001196 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 352 | start GGB234001197 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 353 | start GGB234001200 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 354 | start GGB234001202 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 355 | start GGB234001209 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 356 | start GGB234001211 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 357 | start GGB234001213 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 358 | start GGB234001218 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 359 | start GGB234001219 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 360 | start GGB234001287 public_busroute_info
조회날짜 : 20210920 | loc

조회날짜 : 20210920 | location : 31140 | cnt : 446 | start GGB234001182 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 447 | start GGB234001185 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 448 | start GGB234001188 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 449 | start GGB234001201 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 450 | start GGB234001206 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 451 | start GGB234001220 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 452 | start GGB234001223 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 453 | start GGB234001224 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 454 | start GGB234001284 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 455 | start GGB234001288 public_busroute_info
조회날짜 : 20210920 | location : 31140 | cnt : 456 | start GGB234001462 public_busroute_info
조회날짜 : 20210920 | loc